In [47]:
%reload_ext sql

In [48]:
%sql postgresql://appdev@data/appdev

'Connected: appdev@appdev'

# Materialized Joins

#### 1. On the table "circuits" report:

In [49]:
%sql SELECT * FROM circuits LIMIT 10;

10 rows affected.


circuitid,circuitref,name,location,country,lat,lng,alt,url,position
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,"(144.968,-37.8497)"
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,None,http://en.wikipedia.org/wiki/Sepang_International_Circuit,"(101.738,2.76083)"
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,None,http://en.wikipedia.org/wiki/Bahrain_International_Circuit,"(50.5106,26.0325)"
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,None,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya,"(2.26111,41.57)"
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,None,http://en.wikipedia.org/wiki/Istanbul_Park,"(29.405,40.9517)"
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,None,http://en.wikipedia.org/wiki/Circuit_de_Monaco,"(7.42056,43.7347)"
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,None,http://en.wikipedia.org/wiki/Circuit_Gilles_Villeneuve,"(-73.5228,45.5)"
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,None,http://en.wikipedia.org/wiki/Circuit_de_Nevers_Magny-Cours,"(3.16361,46.8642)"
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,None,http://en.wikipedia.org/wiki/Silverstone_Circuit,"(-1.01694,52.0786)"
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,None,http://en.wikipedia.org/wiki/Hockenheimring,"(8.56583,49.3278)"


In [50]:
%sql select * from pg_indexes where tablename = 'circuits';

3 rows affected.


schemaname,tablename,indexname,tablespace,indexdef
f1db,circuits,idx_17102_url,None,CREATE UNIQUE INDEX idx_17102_url ON circuits USING btree (url)
f1db,circuits,circuits_position_idx,None,"CREATE INDEX circuits_position_idx ON circuits USING gist (""position"")"
f1db,circuits,idx_17102_primary,None,CREATE UNIQUE INDEX idx_17102_primary ON circuits USING btree (circuitid)


- What type of indices exists for the table and why they are of that type (and not some other type)


idx_17102_url index is of btree (url)

BTREE is the default index type for most storage engines. The general idea of a B-Tree is that all the values are stored in order, and each leaf page is the same distance from the root. B Tree index speeds up the data access because storage engine don't have to scan the whole table instead it will start from root node.

circuits_position_idx index is of gist ("position")

GIST allows to generalise the search structures, so we can build indexes on type we choose. In this case GIST index is with <key, pointer> pairs using position instead of integers (like in B-trees).

idx_17102_primary index is of btree (circuitid)

In this case index is of BTREE on circuitid column.

- The amount of space each index takes up

In [51]:
%sql select pg_size_pretty(pg_relation_size('idx_17102_url'));

1 rows affected.


pg_size_pretty
16 kB


In [52]:
%sql select pg_size_pretty(pg_relation_size('circuits_position_idx'));

1 rows affected.


pg_size_pretty
8192 bytes


In [53]:


%sql select pg_size_pretty(pg_relation_size('idx_17102_primary'));



1 rows affected.


pg_size_pretty
16 kB


# The winner of all the races, but only if they are younger than 38 years.

In [54]:
%sql SELECT * FROM information_schema.tables WHERE table_schema = 'f1db';

13 rows affected.


table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
appdev,f1db,circuits,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorresults,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructors,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,constructorstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,drivers,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,driverstandings,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,laptimes,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,pitstops,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,qualifying,BASE TABLE,None,None,None,None,None,YES,NO,None
appdev,f1db,races,BASE TABLE,None,None,None,None,None,YES,NO,None


In [65]:
%%sql
SELECT races.date as "Race Date",
drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38
LIMIT 25;

25 rows affected.


Race Date,Driver's Surname,Driver's Age,Track Time(ms),Name,Circuit Name
2008-03-16,Kovalainen,36.0,04:30:00,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Kovalainen,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2008-08-24,Massa,36.0,12:00:00,European Grand Prix,Valencia Street Circuit
2008-10-12,Massa,36.0,04:30:00,Japanese Grand Prix,Fuji Speedway
2008-10-19,Hamilton,33.0,07:00:00,Chinese Grand Prix,Shanghai International Circuit
2008-11-02,Massa,36.0,17:00:00,Brazilian Grand Prix,Autódromo José Carlos Pace
2007-04-08,Hamilton,33.0,07:00:00,Malaysian Grand Prix,Sepang International Circuit
2007-04-15,Massa,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2007-05-13,Massa,36.0,12:00:00,Spanish Grand Prix,Circuit de Barcelona-Catalunya
2007-05-27,Alonso,36.0,12:00:00,Monaco Grand Prix,Circuit de Monaco


# Describe the query using ``EXPLAIN ANALYZE``

In [56]:
%%sql EXPLAIN ANALYZE
SELECT races.date as "Race Date", 
drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

22 rows affected.


QUERY PLAN
Hash Join (cost=77.92..792.94 rows=86 width=66) (actual time=0.871..4.711 rows=154 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=74.28..787.57 rows=86 width=50) (actual time=0.814..4.539 rows=154 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=34.32..746.53 rows=86 width=19) (actual time=0.431..4.093 rows=154 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=16) (actual time=0.008..3.612 rows=257 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 23420
-> Hash (cost=30.82..30.82 rows=280 width=19) (actual time=0.410..0.410 rows=60 loops=1)



EXPLAIN ANALYZE gives us detailed information about the query plan. It displays the actions executed with the arrows pointing to the information of the action "->". The information display is the actual time, rows and the loops it went through. The actions also include the number of Buckets, Batches and memory being used.

    How many calls are you making?
    There are 10 actions taken on this query.

    How long does it take to perform the query?
    It took 5.428 ms to perform the query.



# Create a materialized view of your query.

In [60]:
%%sql CREATE VIEW winners AS
SELECT races.date as "Race Date", 
drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

(psycopg2.ProgrammingError) relation "winners" already exists
 [SQL: 'CREATE VIEW winners AS\nSELECT races.date as "Race Date", \ndrivers.surname as "Driver\'s Surname", \ndate_part(\'year\', age(\'2018-03-19\',dob)) as "Driver\'s Age", \nraces.time as "Track Time(ms)", \nraces.name as "Name", \ncircuits.name as "Circuit Name" \nFROM results\nJOIN drivers USING (driverid)\nJOIN races USING (raceId)\nJOIN circuits USING (circuitid)\nWHERE results.rank = 1\nAND date_part(\'year\', age(\'2018-03-19\',dob)) < 38;']


In [61]:
%sql EXPLAIN ANALYZE SELECT * FROM winners;

22 rows affected.


QUERY PLAN
Hash Join (cost=77.92..792.94 rows=86 width=66) (actual time=1.090..5.808 rows=154 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=74.28..787.57 rows=86 width=50) (actual time=1.041..5.653 rows=154 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=34.32..746.53 rows=86 width=19) (actual time=0.567..5.130 rows=154 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=16) (actual time=0.009..4.500 rows=257 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 23420
-> Hash (cost=30.82..30.82 rows=280 width=19) (actual time=0.549..0.549 rows=60 loops=1)


In [62]:
%sql CREATE MATERIALIZED VIEW winners_cache AS SELECT * FROM winners;

154 rows affected.


[]

In [63]:
%sql EXPLAIN ANALYZE SELECT * FROM winners_cache;

3 rows affected.


QUERY PLAN
Seq Scan on winners_cache (cost=0.00..10.50 rows=50 width=1568) (actual time=0.010..0.028 rows=154 loops=1)
Planning time: 0.121 ms
Execution time: 0.047 ms


A materialized view is a database object that contains the results of a query. For example, it may be a local copy of data located remotely, or may be a subset of the rows and/or columns of a table or join result, or may be a summary using an aggregate function. Basically, it is a snapshot (which was the actual name in prior releases) which replicates data. The data can not be update following that the execution time is reduced exponentially. The basic use of this would be with databases that are used generally for reading instead of writing.